In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<div dir="rtl">
    در این قسمت کارهای زیر را انجام  می دهیم
    
    * خواندن فایل از پایگاه داده
    * اضافه کردن برچسب مثبت بودن نتیجه تست به داده ها
    * حذف ستون های id و سن به دلیل نیاز نبودن و کامل نبودن
    * حذف سطرهای تکراری از پایگاه داده
</div>


In [9]:
# read in all our data
X_data = pd.read_json("../input/covid-patient-datasets/covid.json")
X_data['result_test'] = 'yes'
#X_data.info()
to_drop=['#','age']
X_data.drop(to_drop,inplace=True,axis=1)
#print(X_data.shape)
X_data.head()
num_iter=X_data.duplicated().sum()
print(f'{num_iter} of data are iterated')
X_data=X_data.drop_duplicates()
display(X_data)

<div dir="rtl">
    در ادامه کارهای زیر را برای مهیا کردن داده‌ها انجام  می‌دهیم
    
    * عددی کردن پایگاه داده با تبدیل مقادیر yes به 1 و no به صفر. لازم به ذکر است به دلیل اینکه در داده‌های ثبت شده اشتباهاتی وجود داشته و مواردی نیز ناهمگونی دیده میشد مجبور به تغییر موارد دیگری نیز شدیم.
    * اضافه کردن مجموعه داده‌های دیگر به صورتی که شرایط عکس داده‌های فعلی را داشته و برچسب نتیجه تست آن نیز منفی باشد.
    * ادغام دو مجموعه و ایجاد دیتاست نهایی
    * نرمال کردن داده با استفاده از کتابخانه sklearn
</div>

In [10]:
from sklearn import preprocessing

X_data.replace(('yes','es', 'Yes', 'no', 'No' , ''), (1, 1, 1, 0, 0 , 0), inplace=True)
Y_data = X_data.copy()
Y_data.replace((1, 0), (0, 1), inplace=True)

data = X_data.append(Y_data, ignore_index=True)
# Randomize the dataset
data = data.sample(frac=1, random_state=1)
feature_vector = X_data.columns
data_normalized = preprocessing.normalize(data)
#display(data)
#display(X_data)
#display(Y_data)
display(data_normalized)

**FIND-S Algorithm**

In [12]:
import numpy as np

d = np.array(data_normalized)[:,:-1]
target = np.array(data_normalized)[:,-1]
#display(d)
#display(target)
# traing function to implement Find-s algorithm
def find_s(c,t):
    for i, val in enumerate(t):
        if val == 1:
            specific_hypothesis = c[i].copy()
            break
             
    for i, val in enumerate(c):
        if t[i] == 1:
            for x in range(len(specific_hypothesis)):
                if val[x] != specific_hypothesis[x]:
                    specific_hypothesis[x] = -1
                else:
                    pass
                 
    return specific_hypothesis


print("The final hypothesis is:",find_s(d,target))

**Candidate Elimination algorithm**

In [14]:
d = np.array(data_normalized)[:,:-1]
target = np.array(data_normalized)[:,-1]

#Candidate Elimination algorithm
def candidate_elimination(concepts, target):
    specific_h = concepts[0].copy()
    #print("Initialization of specific_h and general_h")
    #print("specific_h: ",specific_h)
    general_h = [[-1 for i in range(len(specific_h))] for i in range(len(specific_h))]
    #print("general_h: ",general_h)
    #print("concepts: ",concepts)
    for i, h in enumerate(concepts):
        if target[i] == 1:
            for x in range(len(specific_h)):
                if h[x] != specific_h[x]:
                    specific_h[x] = -1
                    general_h[x][x] = -1
        if target[i] == 0:
            for x in range(len(specific_h)):
                if h[x] != specific_h[x]:
                    general_h[x][x] = specific_h[x]
                else:
                    general_h[x][x] = -1
    #print("\nSteps of Candidate Elimination Algorithm: ",i+1)
    #print("Specific_h: ",i+1)
    #print(specific_h,"\n")
    #print("general_h :", i+1)
    #print(general_h)
    indices = [i for i, val in enumerate(general_h) if val == [-1, -1, -1, -1, -1, -1]]
    #print("\nIndices",indices)
    for i in indices:
        general_h.remove([-1, -1, -1, -1, -1, -1])
    return specific_h, general_h

s_final,g_final = candidate_elimination(d, target)
print("\nFinal Specific_h:", s_final, sep="\n")
print("Final General_h:", g_final, sep="\n")


**Split Train and Test data**

In [16]:
d = np.array(data)[:,:-1]
target = np.array(data)[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(d, target, test_size = 0.30, random_state = 0)

**Naive Bayes Algorithm**

In [24]:
# Naive Bayes Algorithm
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_temp = sc.fit_transform(X_train)
X_test_temp = sc.transform(X_test)

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train_temp, Y_train)

y_pred  =  classifier.predict(X_test_temp)

from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(Y_test, y_pred))
print(classification_report(Y_test,y_pred))

**KNN**

In [20]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, Y_train)
y_pred1 = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test, y_pred1))
print(classification_report(Y_test, y_pred1))

**Decision Tree**

In [31]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,Y_train)

#Predict the response for test dataset
y_pred3 = clf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
# Model Accuracy, how often is the classifier correct?
print(confusion_matrix(Y_test, y_pred3))
print(classification_report(Y_test, y_pred3))

In [35]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)
img=tree.plot_tree(clf)

[...]

**Decision Tree ID3 Algorithm**

In [27]:
import math
from collections import deque

class Node:
    """Contains the information of the node and another nodes of the Decision Tree."""
    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None


class DecisionTreeClassifier:
    """Decision Tree Classifier using ID3 algorithm."""

    def __init__(self, X, feature_names, labels):
        self.X = X
        self.feature_names = feature_names
        self.labels = labels
        self.labelCategories = list(set(labels))
        self.labelCategoriesCount = [list(labels).count(x) for x in self.labelCategories]
        self.node = None
        self.entropy = self._get_entropy([x for x in range(len(self.labels))])  # calculates the initial entropy
        return

    def _get_entropy(self, x_ids):
        """ Calculates the entropy.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        __________
        :return: entropy: float, Entropy.
        """
        # sorted labels by instance id
        labels = [self.labels[i] for i in x_ids]
        # count number of instances of each category
        label_count = [labels.count(x) for x in self.labelCategories]
        # calculate the entropy for each category and sum them
        entropy = sum([-count / len(x_ids) * math.log(count / len(x_ids), 2) if count else 0 for count in label_count])
        return entropy

    def _get_information_gain(self, x_ids, feature_id):
        """Calculates the information gain for a given feature based on its entropy and the total entropy of the system.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        :param feature_id: int, feature ID
        __________
        :return: info_gain: float, the information gain for a given feature.
        """
        # calculate total entropy
        info_gain = self._get_entropy(x_ids)
        # store in a list all the values of the chosen feature
        x_features = [self.X[x][feature_id] for x in x_ids]
        # get unique values
        feature_vals = list(set(x_features))
        # get frequency of each value
        feature_vals_count = [x_features.count(x) for x in feature_vals]
        # get the feature values ids
        feature_vals_id = [
            [x_ids[i]
            for i, x in enumerate(x_features)
            if x == y]
            for y in feature_vals
        ]

        # compute the information gain with the chosen feature
        info_gain = info_gain - sum([val_counts / len(x_ids) * self._get_entropy(val_ids)
                                     for val_counts, val_ids in zip(feature_vals_count, feature_vals_id)])

        return info_gain

    def _get_feature_max_information_gain(self, x_ids, feature_ids):
        """Finds the attribute/feature that maximizes the information gain.
        Parameters
        __________
        :param x_ids: list, List containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        __________
        :returns: string and int, feature and feature id of the feature that maximizes the information gain
        """
        # get the entropy for each feature
        features_entropy = [self._get_information_gain(x_ids, feature_id) for feature_id in feature_ids]
        # find the feature that maximises the information gain
        max_id = feature_ids[features_entropy.index(max(features_entropy))]

        return self.feature_names[max_id], max_id

    def id3(self):
        """Initializes ID3 algorithm to build a Decision Tree Classifier.
        :return: None
        """
        x_ids = [x for x in range(len(self.X))]
        feature_ids = [x for x in range(len(self.feature_names))]
        self.node = self._id3_recv(x_ids, feature_ids, self.node)
        print('')

    def _id3_recv(self, x_ids, feature_ids, node):
        """ID3 algorithm. It is called recursively until some criteria is met.
        Parameters
        __________
        :param x_ids: list, list containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        :param node: object, An instance of the class Nodes
        __________
        :returns: An instance of the class Node containing all the information of the nodes in the Decision Tree
        """
        if not node:
            node = Node()  # initialize nodes
        # sorted labels by instance id
        labels_in_features = [self.labels[x] for x in x_ids]
        # if all the example have the same class (pure node), return node
        if len(set(labels_in_features)) == 1:
            node.value = self.labels[x_ids[0]]
            return node
        # if there are not more feature to compute, return node with the most probable class
        if len(feature_ids) == 0:
            node.value = max(set(labels_in_features), key=labels_in_features.count)  # compute mode
            return node
        # else...
        # choose the feature that maximizes the information gain
        best_feature_name, best_feature_id = self._get_feature_max_information_gain(x_ids, feature_ids)
        node.value = best_feature_name
        node.childs = []
        # value of the chosen feature for each instance
        feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
        # loop through all the values
        for value in feature_values:
            child = Node()
            child.value = value  # add a branch from the node to each feature value in our feature
            node.childs.append(child)  # append new child node to current node
            child_x_ids = [x for x in x_ids if self.X[x][best_feature_id] == value]
            if not child_x_ids:
                child.next = max(set(labels_in_features), key=labels_in_features.count)
                print('')
            else:
                if feature_ids and best_feature_id in feature_ids:
                    to_remove = feature_ids.index(best_feature_id)
                    feature_ids.pop(to_remove)
                # recursively call the algorithm
                child.next = self._id3_recv(child_x_ids, feature_ids, child.next)
        return node

    def printTree(self):
        if not self.node:
            return
        nodes = deque()
        nodes.append(self.node)
        while len(nodes) > 0:
            node = nodes.popleft()
            print(node.value)
            if node.childs:
                for child in node.childs:
                    print('({})'.format(child.value))
                    nodes.append(child.next)
            elif node.next:
                print(node.next)

tree_clf = DecisionTreeClassifier(X=X_train, feature_names=feature_vector[:-1], labels=Y_train)
print("System entropy {:.4f}".format(tree_clf.entropy))

**Find 5 worst features**

In [45]:
resultDicDT = {}
resultDicNB = {}

for i in range(len(X_train[0])):
    X_train_removed_feature = X_train
    X_test_removed_feature = X_test
    for row in X_train_removed_feature:
        np.delete(row,i)
    for row in X_test_removed_feature:
        np.delete(row,i)
  
    # Decision Tree
    # Create Decision Tree classifer object
    clf = DecisionTreeClassifier()
    # Train Decision Tree Classifer
    clf = clf.fit(X_train_removed_feature ,Y_train)
    #Predict the response for test dataset
    y_pred3 = clf.predict(X_test_removed_feature)
    # Model Accuracy, how often is the classifier correct?
    ac = metrics.accuracy_score(Y_test, y_pred3)
    resultDicDT[i] = ac

    # Naive Bayes Algorithm
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train_temp = sc.fit_transform(X_train_removed_feature)
    X_test_temp = sc.transform(X_test_removed_feature)
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train_temp, Y_train)
    y_pred  =  classifier.predict(X_test_temp)
    from sklearn.metrics import confusion_matrix,accuracy_score
    ac = accuracy_score(Y_test,y_pred)
    resultDicNB[i] = ac

def translate(i):
    for f, b in zip(data.columns,range(len(data.columns))):
        if i == b:
          return f

print("\nDT:")  
sort_orders_DT = sorted(resultDicDT.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_DT:
    print("After removing feature", translate(i[0]) ,"Accuracy is:", i[1])

print("\nNB:")
sort_orders_NB = sorted(resultDicNB.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_NB:
    print("After removing feature", translate(i[0]) ,"Accuracy is:", i[1])